# Start

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import pyodbc

from pat_back import PAT_FLAG, VALIDATE_RULE, COVERAGE_TYPE, PERIL_SUBGROUP, DEDDUCT_TYPE, RATING_TYPE, PSOLD_PERSP
from pat_back import PsoldRating, FlsRating, MbRating

fld = r"C:\Users\cxiao\source\repos\property-exposure\gcpropexpo\GCPropExpo\tests\testthat"
df_loc = pd.read_csv(f'{fld}/Inputs/PATLocations.csv')
df_loc['LOCID'] = df_loc.index +1
df_pol = pd.read_csv(f'{fld}/Inputs/PATPolicies.csv')
df_res = pd.read_csv(f'{fld}/Controls/PAT_Ans.csv')

DT = df_pol.merge(df_loc, on='Policy ID')
DT.columns=['PolicyID', 'Limit', 'Retention', 'PolPrem', 'LossRatio', 'Participation', 'TIV', 'Stack', 'RatingGroup', 'LOCID']



### Input Parameters

In [2]:
# test_that("Accuracy", {
#   loc <- fread('./Inputs/PATLocations.csv')
#   pol <- fread('./Inputs/PATPolicies.csv')
#   loc$`Rating Grp`[20] <- 58L
#   covgInfo <- list(subPrem = 5e6,
#                    ratio = 0.8,
#                    adCvg = 0,
#                    dedType = 1L,
#                    curveID = 58L,
#                    b = 0.665396581,
#                    g = 131.7850411,
#                    customType = 1L,
#                    type = 'mb')
#   correctAns <- masterAns[TEST == 'MB1', .(LOCID, PREMIUM)]
#   testResults <- PAT(pol, loc, covgInfo)
#   expect_equal(testResults$allocPrem, correctAns$PREMIUM, tolerance = tol)
# })

ratio = 0.8
covg = COVERAGE_TYPE['Building_Contents_BI']
adCvg = 0
dedType = DEDDUCT_TYPE['Retains_Limit']
curveID = 58
customType = 1
test = 'MB1'
para = {
    'b' : 0.665396581,
    'g' : 131.7850411
}

DT.fillna({'LossRatio': ratio}, inplace=True)

### Curve data

In [23]:
conn_str = f'''DRIVER={{SQL Server}};Server=dfwcat-pm;Database=premium_allocation_tool;
            User Id=sa;Password=In5tr@t1;MultipleActiveResultSets=true;'''
with pyodbc.connect(conn_str) as conn:
     df_mb = pd.read_sql_query(f"""select ID, b, g, cap from mb_curves order by ID""", conn).set_index('ID')

if para: 
     df_mb.loc[curveID] = df_mb.loc[curveID].to_dict() | para

In [27]:
p ={}
if p:
    print('aaa')

### Allocation

In [ ]:
rating_model = MbRating(curveID, df_mb, **para)
DT = rating_model.calculate_las(DT, addt_cvg = adCvg, ded_type = dedType.name, custom_type = customType )

DT['Premium'] = (DT.PolLAS-DT.DedLAS) * DT.LossRatio 
sumLAS = DT.groupby('PolicyID').agg( 
    {'Premium': 'sum'}).rename(columns={'Premium': 'sumLAS'})
DT = DT.merge(sumLAS, on='PolicyID')
DT['Premium'] *= DT['PolPrem'] / DT['sumLAS']
DT.fillna({'Premium':0},inplace=True)


### Compare

In [ ]:
df = df_res[df_res.TEST== test].merge(DT, on ='LOCID')[['LOCID','RatingGroup', 'PREMIUM','Premium']]
df['Diff']= np.abs(df.PREMIUM - df.Premium)/df.PREMIUM
print('Accuracy: ', df.Diff.max())
df